## Группа DS03-onl



Студент Парфимович Алексей

## Домашнее задание №23

Задача поиска и индентефикации лиц.
- Определить координаты зрачка.


### !!! Перед началом работы необходимо распаковать архив shape_predictor_68_face_landmarks.zip в каталоге \assets !!!

См. пример реализации алгоритма по ссылке
https://habr.com/ru/post/429024/  

In [17]:
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2


# параметры конфигурации
config = {
    "face_detector_prototxt": "assets/deploy.prototxt",
	"face_detector_weights": "assets/res10_300x300_ssd_iter_140000.caffemodel",
	"landmark_predictor": "assets/shape_predictor_68_face_landmarks.dat",
    "sunglasses": "assets/sunglasses.png",
	"sunglasses_mask": "assets/sunglasses_mask.png",
    "min_confidence": 0.5
}

# Загружаем в память детектор лиц DNN OpenCV и предиктор ориентиров лица dlib 
# для локализации отдельных структур лица (глаза, брови, нос, рот, линию подбородка)
detector = cv2.dnn.readNetFromCaffe(config["face_detector_prototxt"], config["face_detector_weights"])
predictor = dlib.shape_predictor(config["landmark_predictor"])

#### Реализуем функции для наложения темных очков на исходное изображение:

 Функция альфа-смешивания _alpha_blend_ преобразует передний план, фон и альфа-канал в числа с плавающей запятой в диапазоне [0, 1], выполняет альфа-смешивание, добавляет передний план и фон, возврашает результат в точку вызова функции

In [18]:
def alpha_blend(fg, bg, alpha):
	# преобразуем фон, передний план и альфа-канал в числа с плавающей запятой в диапазоне [0, 1]
	fg = fg.astype("float")
	bg = bg.astype("float")
	alpha = alpha.astype("float") / 255
 
	# выполняем альфа-смешивание
	fg = cv2.multiply(alpha, fg)
	bg = cv2.multiply(1 - alpha, bg)
 
	# добавляем передний план и фон, получая конечный результат
	output = cv2.add(fg, bg)
	
	# возвращаем результат
	return output.astype("uint8")

Функция _overlay_image_ накладывает передний план (fg) на верхнюю часть фонового изображения (bg) по координатам coords (координаты (x, y)), реализуя альфа-прозрачность по маске переднего плана fgMask

In [19]:
def overlay_image(bg, fg, fgMask, coords):
	# определяем размер переднего плана (ширина, высота) и координаты его размещения
	(sH, sW) = fg.shape[:2]
	(x, y) = coords
 
	# наложение должно быть точно такой ширины и высоты как исходная картинка, 
	# но полностью пустым, кроме переднего плана, который мы добавляем
	overlay = np.zeros(bg.shape, dtype="uint8")
	overlay[y:y + sH, x:x + sW] = fg
 
	# альфа-канал контролирует, *координаты* и *степень* прозрачности, 
	# его размеры такие же, как у исходного изображения, 
	# но он содержит только маску наложения
	alpha = np.zeros(bg.shape[:2], dtype="uint8")
	alpha[y:y + sH, x:x + sW] = fgMask
	alpha = np.dstack([alpha] * 3)
 
	# выполняем альфа-смешивание для переднего плана, фона и альфа-канала
	output = alpha_blend(overlay, bg, alpha)
 
	# возвращаем результат обработки
	return output

Функция _put_on_sunglasses_ реализует функционал обнаружения и выделения лица на изображении,  
локализует ориентиры лица, определяет координаты центров глаз выделенного лица и угол между центроидами глаз,
вычисляет требуемые поворот и изменение размеров солнечных очков для наложения на лицо,
выполняет наложение солнечных очков (sunglasses) на исходное изображение.

In [20]:
def put_on_sunglasses(image, sg, sgMask):

	'''
	Выполняем обнаружение лица:
	- конструируем из исходного изображения blob для отправки в детектор лиц нейросети
	- выполняем процедуру обнаружения лиц.
	- Находим лицо с наибольшим значением вероятности и сравниваем его с минимально допустимым порогом вероятности. Если критерии не выполняются - завершаем работу, иначе продолжаем.
	'''

	(H, W) = image.shape[:2]

	# создаём blob для отправки в детектор лиц нейросети
	blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
		(300, 300), (104.0, 177.0, 123.0))
	
	# передаём блоб в нейросеть и получаем результаты
	detector.setInput(blob)
	detections = detector.forward()
	
	# для наложения очков нужно только одно лицо, поэтому
	# определяем лицо, для которого выдаётся максимальная вероятность
	i = np.argmax(detections[0, 0, :, 2])
	confidence = detections[0, 0, i, 2]
	
	# отфильтровываем слабые результаты
	if confidence < config["min_confidence"]:
		print("[INFO] no reliable faces found")
		sys.exit(0)

	'''
	Извлекаем обнаруженное лицо и вычисляем ориентиры:
	- извлекаем координаты ограничительной рамки вокруг лица.
	- создаём объект rectangle в dlib и применяем локализацию ориентиров лица.
	- извлекаем (x, y)-координаты leftEyePts и rightEyePts, соответственно
	'''

	# вычисляем координаты (x, y) ограничительной рамки на лице
	box = detections[0, 0, i, 3:7] * np.array([W, H, W, H])
	(startX, startY, endX, endY) = box.astype("int")
	
	# конструируем прямоугольный объект dlib из координат ограничительной
	# рамки и определяем ориентиры внутри него
	rect = dlib.rectangle(int(startX), int(startY), int(endX), int(endY))
	shape = predictor(image, rect)
	shape = face_utils.shape_to_np(shape)
	
	# берём индексы ориентиров для левого и правого глаз, вычисляем координаты каждого глаза
	(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
	(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
	leftEyePts = shape[lStart:lEnd]
	rightEyePts = shape[rStart:rEnd]

	'''
	По заданным координатам глаз рассчитываем, где и как размещать солнцезащитные очки:
	- вычисляем центр каждого глаза и затем угол между центроидами
	- выполняем поворот и изменение размера очков (используем функцию rotate_bound чтобы OpenCV не обрезал части, которые не видны после аффинного преобразования)
	- применяем аналогичные операции к маске, предварительно выполнив преобразование в оттенки серого и бинаризацию (маски всегда бинарны)
	'''

	# вычисляем центроиды для каждого глаза
	leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
	rightEyeCenter = rightEyePts.mean(axis=0).astype("int")
	
	# вычисляем угол между центроидами глаз
	dY = rightEyeCenter[1] - leftEyeCenter[1]
	dX = rightEyeCenter[0] - leftEyeCenter[0]
	angle = np.degrees(np.arctan2(dY, dX)) - 180
	
	# поворачиваем изображение очков на вычисленный угол, чтобы поворот очков соответствовал наклону головы
	sg = imutils.rotate_bound(sg, angle)
	
	# очки не должны покрывать всю ширину лица, а только глаза 
	# выполняем примерную оценку и указываем 90% ширины лица в качестве ширины очков
	sgW = int((endX - startX) * 0.9)
	sg = imutils.resize(sg, width=sgW)
	
	# в очках есть прозрачные части (нижняя часть, под линзами и носом), 
	# для получения приемлемого результатаm применяем маску и альфа-смешивание
	sgMask = cv2.cvtColor(sgMask, cv2.COLOR_BGR2GRAY)
	sgMask = cv2.threshold(sgMask, 0, 255, cv2.THRESH_BINARY)[1]
	sgMask = imutils.rotate_bound(sgMask, angle)
	sgMask = imutils.resize(sgMask, width=sgW, inter=cv2.INTER_NEAREST)

	# Формруем конечное изображение
	shiftX = int(sg.shape[1] * 0.25)
	shiftY = int(sg.shape[0] * 0.35)
	y = max(0, rightEyeCenter[1] - shiftY)
	
	# Накладываем солнечные очки (sunglasses) на исходное изображение
	output = overlay_image(image, sg, sgMask, (rightEyeCenter[0] - shiftX, y))

	return output

In [21]:
# загружаем очки и соответствующую маску
sg = cv2.imread(config["sunglasses"])
sgMask = cv2.imread(config["sunglasses_mask"])

cap = cv2.VideoCapture(0)

while True:
    # cap возвращает 2 параметра: ret - bool (true, если сигнал с камеры поступает), frame - изображение
    ret, frame = cap.read()
    
    if not ret: break

    # выполняем наложение солнечных очков
    output = put_on_sunglasses(frame, sg, sgMask)

    cv2.imshow('output', output)

    # Завершить работу по нажатию клавиши "q" (!!! Обязательна английская раскладка)
    if cv2.waitKey(10) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()